# RoadSafety

## Importing Modules

Importing libraries installed using PyPI and functions present in scripts created in for this project.

In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
from google.protobuf import text_format

from joblib import Parallel, delayed
import joblib
import multiprocessing

import time

## Executing Pre-Trained TensorFlow Model

In [9]:
def load_graph(frozen_graph_filename):
    
    """
    Args:
        frozen_graph_filename (str): Full path to the .pb file.
    """
    # We load the protobuf file from the disk and parse it to retrieve the unserialized graph_def
    with tf.io.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        
        #text_format.Merge(frozen_graph_filename,graph_def)

    # Then, we import the graph_def into a new Graph and returns it
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
        print(graph)
        return graph

def segment(image_file):
    """
    Does the segmentation on the given image.
    Args:
        graph (Tensorflow Graph)
        image_file (str): Full path to your image
    Returns:
        segmentation_mask (np.array): The segmentation mask of the image.
    """
    # We access the input and output nodes

    #graph = load_graph(MODEL_FILE)

    tf.reset_default_graph()

    MODEL_FILE = 'cityscapes/frozen_inference_graph.pb'
    graph = load_graph(MODEL_FILE)

    x = graph.get_tensor_by_name('prefix/ImageTensor:0')
    y = graph.get_tensor_by_name('prefix/SemanticPredictions:0')

    # We launch a Session
    with tf.compat.v1.Session(graph=graph) as sess:
        image = Image.open(image_file)
        image = image.resize((513, 513)) # 299 513
        image_array = np.array(image)
        image_array = np.expand_dims(image_array, axis=0)

        # Note: we don't nee to initialize/restore anything
        # There is no Variables in this graph, only hardcoded constants
        pred = sess.run(y, feed_dict={x: image_array})

        pred = pred.squeeze()
        
        print(pred)

    return pred

MODEL_FILE = 'cityscapes/frozen_inference_graph.pb'
graph = load_graph(MODEL_FILE)

x = graph.get_tensor_by_name('prefix/ImageTensor:0')
y = graph.get_tensor_by_name('prefix/SemanticPredictions:0')

if __name__ == '__main__':

    N_CLASSES = 150 # 30
    IMAGE_FILE = 'img/london4.jpg'
'''
    for image in images:

        startTime = time.time()
        prediction = segment(IMAGE_FILE)
        print(startTime - time.time())
        #print('london-images-new/' + image)

        #prediction = segment(graph, 'img2/' + image)
'''
'''

    prediction = segment(graph, IMAGE_FILE)
    segmented_image = parse_pred(prediction, N_CLASSES)

    segmented_image.show()
'''

def auxx11 (image_var):

    output_aux = segment('img2/' + image_var)

    return output_aux

num_cores = multiprocessing.cpu_count()

print(num_cores)

out_output = Parallel(n_jobs=2)(delayed(auxx11)(image) for image in images)

8


AttributeError: module 'tensorflow' has no attribute 'reset_default_graph'

In [6]:
!conda list

# packages in environment at /Users/luisrita/opt/anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
_tflow_select             2.3.0                       mkl  
absl-py                   0.9.0                    py37_0  
alabaster                 0.7.12                   py37_0  
anaconda                  2019.10                  py37_0  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.7                    py37_0  
anaconda-project          0.8.3                      py_0  
appnope                   0.1.0                    py37_0  
appscript                 1.1.0            py37h1de35cc_0  
asn1crypto                1.0.1                    py37_0  
astor                     0.8.0                    py37_0  
astroid                   2.3.1                    py37_0  
astropy                   3.2.2            py37h1de35cc_0  
atomicwrites              1.3.0

In [2]:
import os

images = []

for root, dirs, files in os.walk("./img2"):
    for filename in files[0:4]:

        images.append(filename)
        #print(filename)

print(images)

['london.png', 'london2.png', 'london - cópia.png', 'london2 - cópia.png']


In [13]:
from shutil import copyfile
import os

copyfile("./img2/london.png", "./img/london(copy2).png")

'./img/london(copy2).png'

Joining Detected Objects Text Files

In [24]:
import os

string = ""

excluded_list = {}

files = [f for f in os.listdir('./yolotemp') if f != ".DS_Store" and f != "new"]

for filename1 in files:

    if filename1.split('_')[0] not in excluded_list:

        text_file = open('./yolotemp/new/' + filename1.split('_')[0] + ".txt", "w")
        #text_file.write("")
        excluded_list[filename1.split('_')[0]] = 0

    string = ""

    for filename2 in files:

        if filename1.split('_')[0] == filename2.split('_')[0] and excluded_list[filename1.split('_')[0]] != 4:

            string = string + '\n' + open('./yolotemp/' + filename2, "r").read()

            excluded_list[filename1.split('_')[0]] = excluded_list[filename1.split('_')[0]] + 1

    text_file.write(string)

In [17]:
file = open('./yolotemp/exp1.txt', 'r')
hii = file.read()

hiii = hii + '\n' + "sssss"

text_file = open("./yolotemp/exp11.txt", "w")
text_file.write(hiii)
text_file.close()

In [23]:
import os, glob
for filename in glob.glob("yolotemp2/111_a.txt"):
    os.remove(filename)

Select 1 Random Image per LSOA

In [3]:
import pandas as pd

data = pd.read_csv (r'image_labels.csv')
#print (df)

df = pd.DataFrame(data, columns= ['lsoa11','img_id'])
print(df)

df_i = df.set_index('img_id')
print(df_i)

lsoa_list = df_i.lsoa11.unique()
print(lsoa_list[0])

img_ids = []
print(len(lsoa_list))
for lsoa_index in range(0, len(lsoa_list)):
    img_ids.append(df_i.index[df['lsoa11'] == lsoa_list[lsoa_index]].tolist()[0])

#print(img_ids)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


           lsoa11  img_id
0       E01000677       1
1       E01000675       5
2       E01000674       8
3       E01000677      10
4       E01000675      15
...           ...     ...
128199  E01002438  132465
128200  E01002438  132467
128201  E01002438  132468
128202  E01002438  132469
128203  E01000290  132470

[128204 rows x 2 columns]
           lsoa11
img_id           
1       E01000677
5       E01000675
8       E01000674
10      E01000677
15      E01000675
...           ...
132465  E01002438
132467  E01002438
132468  E01002438
132469  E01002438
132470  E01000290

[128204 rows x 1 columns]
E01000677
4833


Generating Text File w/ 1 Image per LSOA

In [6]:
string = ""

text_file = open('./yolotemp/list_images.txt', "w")

for img_id in img_ids:

    string = string + '\n' + str(img_id) + "_a" + ".txt"

text_file.write(string)
text_file.close()

Selecting Text Files w/ Matching Name

In [ ]:
import os

images = []

for 

for root, dirs, files in os.walk("./img2"):
    for filename in files[0:4]:

        images.append(filename)
        #print(filename)

print(images)

In [1]:
from shutil import copyfile
import os

right_files = []
all_files = []

with open('yolotemp/list_images.txt') as fp:
   line = fp.readline()
   while line:
       line = fp.readline()
       right_files.append((line.split("\n")[0]).split(".")[0] + ".png")
print(right_files)

for root, dirs, files in os.walk("yolotemp"):
    for filename in files:
        all_files.append(filename)

for file in right_files:
    for filename in all_files:
        if filename == file:
            copyfile("yolotemp/" + filename, "yolotemp/new2/" + filename)

['5_a.png', '8_a.png', '35_a.png', '65_a.png', '66_a.png', '73_a.png', '77_a.png', '99_a.png', '101_a.png', '190_a.png', '203_a.png', '234_a.png', '243_a.png', '303_a.png', '342_a.png', '358_a.png', '394_a.png', '416_a.png', '458_a.png', '494_a.png', '545_a.png', '560_a.png', '562_a.png', '578_a.png', '594_a.png', '598_a.png', '635_a.png', '641_a.png', '656_a.png', '688_a.png', '720_a.png', '728_a.png', '743_a.png', '784_a.png', '802_a.png', '804_a.png', '807_a.png', '811_a.png', '812_a.png', '831_a.png', '928_a.png', '942_a.png', '943_a.png', '944_a.png', '950_a.png', '992_a.png', '1008_a.png', '1029_a.png', '1046_a.png', '1072_a.png', '1087_a.png', '1120_a.png', '1187_a.png', '1189_a.png', '1194_a.png', '1233_a.png', '1256_a.png', '1262_a.png', '1281_a.png', '1311_a.png', '1341_a.png', '1375_a.png', '1407_a.png', '1410_a.png', '1414_a.png', '1429_a.png', '1431_a.png', '1607_a.png', '1643_a.png', '1742_a.png', '1743_a.png', '1818_a.png', '1819_a.png', '1823_a.png', '1825_a.png', '1838

Converting Image IDs into LSOA

In [1]:
import pandas as pd

data = pd.read_csv (r'image_labels.csv')

df = pd.DataFrame(data, columns= ['lsoa11','img_id'])

df_i = df.set_index('img_id')

print(df_i.loc[8]) # Uses index img_id
#print(df_i.iloc[8]) # Uses iterative variable

lsoa11    E01000674
Name: 8, dtype: object


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:

def load_graph(frozen_graph_filename):

    """
    Args:
        frozen_graph_filename (str): Full path to the .pb file.
    """
    # We load the protobuf file from the disk and parse it to retrieve the unserialized graph_def
    with tf.io.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

        #text_format.Merge(frozen_graph_filename,graph_def)

    # Then, we import the graph_def into a new Graph and returns it
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
        print(graph)
        return graph

def segment(image_file):
    """
    Does the segmentation on the given image.
    Args:
        graph (Tensorflow Graph)
        image_file (str): Full path to your image
    Returns:
        segmentation_mask (np.array): The segmentation mask of the image.
    """

    # We launch a Session
    with tf.compat.v1.Session(graph=graph) as sess:
        image = Image.open(image_file)
        image = image.resize((513, 513)) # 299 513
        image_array = np.array(image)
        image_array = np.expand_dims(image_array, axis=0)

        # Note: we don't nee to initialize/restore anything
        # There is no Variables in this graph, only hardcoded constants
        pred = sess.run(y, feed_dict={x: image_array})

        pred = pred.squeeze()

        print(pred)

    return pred

MODEL_FILE = 'cityscapes/frozen_inference_graph.pb'
graph = load_graph(MODEL_FILE)

x = graph.get_tensor_by_name('prefix/ImageTensor:0')
y = graph.get_tensor_by_name('prefix/SemanticPredictions:0')

if __name__ == '__main__':

    N_CLASSES = 150 # 30
    IMAGE_FILE = 'img/london4.jpg'
'''
    for image in images:

        startTime = time.time()
        prediction = segment(IMAGE_FILE)
        print(startTime - time.time())
        #print('london-images-new/' + image)

        #prediction = segment(graph, 'img2/' + image)
'''
'''

    prediction = segment(graph, IMAGE_FILE)
    segmented_image = parse_pred(prediction, N_CLASSES)

    segmented_image.show()
'''

for image in images:

    output_aux = segment('img2/' + image)

[[ 2  2  2 ... 10 10 10]
 [ 2  2  2 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 ...
 [ 1  1  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]]
[[10 10 10 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
[[ 2  2  2 ... 10 10 10]
 [ 2  2  2 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 ...
 [ 1  1  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]]
[[10 10 10 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [ ]:
import geopandas


